In [1]:
import requests
import time
import os
import json
import glob
from bs4 import BeautifulSoup

In [2]:
data_path = "/Data"
main_url = "https://www.justice.gov"
page_url = "https://www.justice.gov/news?f[0]=type%3Apress_release&page="

In [3]:
def page_link(page_num):
    return page_url + str(page_num)

In [5]:
def page_content(page_url):
    return requests.get(page_url).content

In [24]:
def get_page_content(page):
    return page_content(page_link(page))

In [25]:
def get_page_data(page):
    return BeautifulSoup(get_page_content(page), 'lxml')

In [26]:
def get_total_pages():
    first_page_data = get_page_data(0)
    total_pages = int(first_page_data.find('a', {'title': 'Go to last page'})['href'].split('=')[2])
    return total_pages

In [30]:
def generate_url_for_page(page):
    page_data = get_page_data(page)
    releases = page_data.find_all('div', {'class': 'views-field views-field-title'})
    urls = []
    for release in releases:
        url = release.find('a')['href']
        if not url in urls:
            urls.append(url)
    return urls

In [42]:
def generate_all_urls(total_pages):
    urls = []
    for i in range(total_pages):
        page_urls = generate_url_for_page(i)
        urls += page_urls
    return urls

In [40]:
def extract_news(urls):
    series_data = []
    for index,url in enumerate(urls):
        if (index%100 == 0):
            print(index)
        news_series = []
        news_url = main_url + url
        news_data = BeautifulSoup(page_content(news_url), 'lxml')
        news_title = news_data.find('h1', {'id': 'node-title'}).text
        news_text = ' '.join([p.text for p in news_data.find('div', {'class' : "field field--name-field-pr-body field--type-text-long field--label-hidden"}).find_all(['p', 'li'])])
        news_date = news_data.find('span', {'class': 'date-display-single'})['content']
        topics_content = news_data.find('div', {'class': 'field field--name-field-pr-topic field--type-taxonomy-term-reference field--label-above'})
        news_topics = []
        if (topics_content):
            even_topics = topics_content.find_all('div',{'class': 'field__item even'})
            odd_topics = topics_content.find_all('div',{'class': 'field__item odd'})
            news_topics = [even_topic.text for even_topic in even_topics] + [odd_topic.text for odd_topic in odd_topics]
        news_series.append(news_date)
        news_series.append(news_title)
        news_series.append(news_text)
        news_series.append(news_topics)
        series_data.append(news_series)
    return series_data

In [43]:
all_urls = generate_all_urls(get_total_pages())

In [47]:
series_data = extract_news(all_urls)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700


In [48]:
import pandas as pd

In [49]:
df = pd.DataFrame(series_data, columns = ['Date', 'Title', 'Text', 'Topics'])

In [50]:
df

,Date,Title,Text,Topics
0,2022-02-11T00:00:00-05:00,Former Queens Business Owner Pleads Guilty to ...,The former owner of a drywall business in Quee...,[Tax]
1,2022-02-11T00:00:00-05:00,Bagel Company Owner Pleads Guilty to Tax Evasi...,A New York man pleaded guilty today to tax eva...,[Tax]
2,2022-02-11T00:00:00-05:00,Northern Virginia Company Settles False Claims...,"Zen Solutions Inc., a Virginia-based company, ...","[Coronavirus, False Claims Act]"
3,2022-02-11T00:00:00-05:00,Medical Director Convicted in $110 Million Add...,A federal jury convicted a Florida doctor yest...,[Health Care Fraud]
4,2022-02-10T00:00:00-05:00,Kansas Man Pleads Guilty to Racially Motivated...,The Justice Department announced today that Co...,"[Civil Rights, Hate Crimes]"
...,...,...,...,...
17795,2009-01-08T00:00:00-05:00,Medical Clinic Owner Pleads Guilty to Role in ...,WASHINGTON – The owner and operator of two M...,[]
17796,2009-01-07T00:00:00-05:00,Justice Department Seeks Injunction \r\nAgains...,WASHINGTON - The Justice Department today fi...,[]
17797,2009-01-07T00:00:00-05:00,Settlement Reached in Virginia Stream \r\nand ...,WASHINGTON—Five defendants associated with t...,[]
17798,2009-01-07T00:00:00-05:00,"Three Staten Island, NY Men Indicted\r\non Fed...",WASHINGTON – The Department of Justice annou...,[]


In [52]:
df.to_csv('raw_data.csv')